<a href="https://colab.research.google.com/github/Ibrah-N/University-Study/blob/main/Ibrhaim_BSDS_A_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment : 01
- Title           : End-To-End Pipelining
- Submitted By    : Muhammad Ibrahim
- Submitted To    : Prof Adnan Amin
- SubmissionDate  : 20-April-2025


# Assignments Task List
  1. Download Dataset
  2. Understand The Dataset
  3. Data Preprocessing
    - 3.1 Data Analysis
    - 3.2 Data Visualization
  4. Model Building Without Feature Engineering
  5. Apply Feature Engineering & Balancing Technique
  6. Retrain The Same Models


# Assignment Tasks

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Given Preprocessing Script

In [ ]:
import mne
import matplotlib.pyplot as plt
from datetime import datetime, date, time, timedelta
from Seizure_times import *
import numpy as np
from sklearn.model_selection import train_test_split
import collections
import os

CPS_seizures = []
elec_seizures = []
visual_seizures = []
noc_seizures = []
normals = []
labels = []
index_vect = []

for patient in [15, 14, 13, 12, 11, 10]:

    if patient == 10:
        files_list=["Record1.edf","Record2.edf"]
    elif patient in (15, 13, 11):
        files_list=["Record1.edf","Record2.edf","Record3.edf","Record4.edf"]
    elif patient in (14, 12):
        files_list=["Record1.edf","Record2.edf","Record3.edf"]


    for file_id, file in enumerate(files_list):
        file=os.path.join("./edfs", "p"+str(patient)+"_"+file)
        data = mne.io.read_raw_edf(file, preload=True)

        seizure_record = []
        index_mat = np.array([0,0])
        ###################### Ordering channels ######################
        if patient in (15, 14, 13, 12, 11):
            data.drop_channels(['EEG Cz-Ref', 'EEG Pz-Ref', 'ECG EKG', 'Manual']) # see attached word file for details
        elif patient == 10:
            data.reorder_channels(['EEG Fp2-Ref', 'EEG Fp1-Ref', 'EEG F8-Ref', 'EEG F4-Ref', 'EEG Fz-Ref', 'EEG F3-Ref', 'EEG F7-Ref', 'EEG A2-Ref', 'EEG T4-Ref', 'EEG C4-Ref', 'EEG C3-Ref', 'EEG T3-Ref', 'EEG A1-Ref', 'EEG T6-Ref', 'EEG P4-Ref', 'EEG P3-Ref', 'EEG T5-Ref', 'EEG O2-Ref', 'EEG O1-Ref'])
        print('#'*150)
        print('#'*150)
        print('#'*150)
        #print(data.info["ch_names"])
        print('#'*150)
        print('#'*150)
        print('#'*150)
        raw_data = data.get_data()      # ndarray 19 x ~5M
        raw_data = np.array(raw_data)
        print(raw_data.shape)
        print('#'*150)
        print('#'*150)
        print('#'*150)
        ###################################################
        record_time = data.info['meas_date']
        record_time = record_time.time()  #File time onset
        record_time = datetime.combine(date.today(), record_time)
        # get seizure time and duration

        r = 'seizures_' + str(patient)
        for i in  range(len(eval(r)[file_id+1])):  #file_id correspond to record number so here we are checking how many
            # seizures inside a specific record
            s_time = time(eval(r)[file_id+1][i][0], eval(r)[file_id+1][i][1], eval(r)[file_id+1][i][2])
            seizure_duration = eval(r)[file_id+1][i][3]

            print('*'*150)
            print("File ID: {}, Record time:{}, Seizure time: {}, Seizure duration: {}".format('patient_' + str(patient) + '_Record'+str(file_id+1)+'_sz'+str(i+1), record_time, s_time, seizure_duration))
            print('*'*150)
            print('*'*150)
            print('*'*150)
            diff = datetime.combine(date.today(), s_time) - record_time # assigned date does not matter
            s_index = int( diff.total_seconds() * 500 ) # 500 sampling rate,
            #s_index: seizure start in samples
            s_index_end = s_index + int(seizure_duration * 500)
            #s_index_end: seizure end in sample
            point_duration = 1  # how many seconds is one point
            #print(diff.days)
            #print(diff.seconds)
            #print(s_index)
            #print(s_index_end)
            print('*'*150)
            print('*'*150)
            print('*'*150)

            # get seizure index

            st = raw_data[:, s_index:s_index_end]

            index_mat = np.vstack([index_mat,[s_index,s_index_end]])

            print(index_mat)

            if seizure_record == []:
                seizure_record = st
            else:
                seizure_record = np.concatenate((seizure_record,st),axis=1)
                #for patient 10, record2, the seizure is 305 sec, 305*500 = 152500

            if len(index_mat) == 2:
                normal_record = np.delete(raw_data, np.s_[index_mat[1,0]:index_mat[1,1]], axis=1) # remove all data corresponding to a seizure


            elif len(index_mat) == 3:
                x1 = np.linspace(index_mat[1,0], index_mat[1,1], index_mat[1,1]-index_mat[1,0]+1, endpoint=True)
                x2 = np.linspace(index_mat[2,0], index_mat[2,1], index_mat[2,1]-index_mat[2,0]+1, endpoint=True)
                y = np.concatenate((x1,x2))
                normal_record = np.delete(raw_data, np.s_[y], axis=1)

            elif len(index_mat) == 4:
                x1 = np.linspace(index_mat[1,0], index_mat[1,1], index_mat[1,1]-index_mat[1,0]+1, endpoint=True)
                x2 = np.linspace(index_mat[2,0], index_mat[2,1], index_mat[2,1]-index_mat[2,0]+1, endpoint=True)
                x3 = np.linspace(index_mat[3,0], index_mat[3,1], index_mat[3,1]-index_mat[2,0]+1, endpoint=True)
                y = np.concatenate((x1,x2,x3))
                normal_record = np.delete(raw_data, np.s_[y], axis=1)

            elif len(index_mat) == 5:
                x1 = np.linspace(index_mat[1,0], index_mat[1,1], index_mat[1,1]-index_mat[1,0]+1, endpoint=True)
                x2 = np.linspace(index_mat[2,0], index_mat[2,1], index_mat[2,1]-index_mat[2,0]+1, endpoint=True)
                x3 = np.linspace(index_mat[3,0], index_mat[3,1], index_mat[3,1]-index_mat[2,0]+1, endpoint=True)
                x4 = np.linspace(index_mat[4,0], index_mat[4,1], index_mat[4,1]-index_mat[2,0]+1, endpoint=True)
                y = np.concatenate((x1,x2,x3,x4))
                normal_record = np.delete(raw_data, np.s_[y], axis=1)

            print(seizure_record.shape)
            print('*'*150)
            print('*'*150)
            print('*'*150)

            if patient == 10:
                for i in range(seizure_duration):
                    data_point = seizure_record[:, i * 500:(i + 1) * 500]  #this will be 19x500
                    elec_seizures.append(data_point)  # this will a list of for example 305 matrix of size 19x500
            elif patient ==13 and file_id<3:
                for i in range(seizure_duration):
                    data_point = seizure_record[:, i * 500:(i + 1) * 500]  #this will be 19x500
                    noc_seizures.append(data_point)  # this will a list of for example 305 matrix of size 19x500
            else:
                for i in range(seizure_duration):           # i is the one second interval
                    data_point = seizure_record[:, i*500:(i+1)*500]
                    CPS_seizures.append(data_point)         # CPS is set of points, each of which is 19x500

            # normals
            for i in range(seizure_duration): #take same number of normal points
                data_point = normal_record[:, i*500:(i+1)*500]
                normals.append(data_point)


# transpose second and third dim
CPS_seizures = np.array(CPS_seizures)
scaler = np.amax(abs(CPS_seizures))
CPS_seizures = CPS_seizures/scaler

elec_seizures = np.array(elec_seizures)
scaler = np.amax(abs(elec_seizures))
elec_seizures = elec_seizures/scaler

noc_seizures = np.array(noc_seizures)
scaler = np.amax(abs(noc_seizures))
noc_seizures = noc_seizures/scaler

normals = np.array(normals)
scaler = np.amax(abs(normals))
normals = normals/scaler

# plt.figure('Normal')
# plt.plot(normals[100].T)
# plt.figure('Seizure')
# plt.plot(seizures[100].T)
# plt.show()
#

x = np.vstack((noc_seizures, elec_seizures, CPS_seizures, normals)) #trying to construct the x #points x 19 x 500

# below order matters
labels.extend([3 for i in range(len(noc_seizures))])
labels.extend([2 for i in range(len(elec_seizures))])
labels.extend([1 for i in range(len(CPS_seizures))])
labels.extend([0 for i in range(len(normals))])

x_train, x_test, y_train, y_test = train_test_split(x, labels, test_size=0.1, random_state=1)

np.save("./data_aligned/x_train", x_train)
np.save("./data_aligned/x_test", x_test)
np.save("./data_aligned/y_train", y_train)
np.save("./data_aligned/y_test", y_test)

##############################################################################
import numpy as np
import scipy.io
matrix = np.load('./data_aligned/x_test.npy')
scipy.io.savemat('./data_aligned/x_test.mat', dict(x_test=matrix))
##############################################################################
matrix2 = np.load('./data_aligned/x_train.npy')
scipy.io.savemat('./data_aligned/x_train.mat', dict(x_train=matrix2))
##############################################################################
matrixy = np.load('./data_aligned/y_test.npy')
scipy.io.savemat('./data_aligned/y_test.mat', dict(y_test=matrixy))
##############################################################################
matrixy2 = np.load('./data_aligned/y_train.npy')
scipy.io.savemat('./data_aligned/y_train.mat', dict(y_train=matrixy2))

## Preprocessing Data

In [1]:
import numpy as np
import os

In [2]:
!cp -r /content/drive/MyDrive/data_aligned /content/

print(os.listdir("/content/data_aligned/"))

['x_test.npy', 'x_train.npy', 'y_train.npy', 'y_test.npy', 'x_test.mat', 'x_train.mat', 'y_test.mat', 'y_train.mat']


In [3]:
x_train = np.load("/content/data_aligned/x_train.npy")
y_train = np.load("/content/data_aligned/y_train.npy")
x_test = np.load("/content/data_aligned/x_test.npy")
y_test = np.load("/content/data_aligned/y_test.npy")

In [4]:
x_train.shape

(6232, 19, 500)

In [5]:
import numpy as np
from scipy.io import loadmat
import pandas as pd
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Dropout,Normalization,BatchNormalization,LayerNormalization
from tensorflow.keras.layers import LSTM,GRU,Embedding,Reshape
from tensorflow.keras.layers import Conv2D,MaxPooling2D,AveragePooling2D,Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam,Adamax
from tensorflow.keras.regularizers import l1,l2

In [6]:
# Allocate 10% of train data to validation set
x_val = x_train[-701:,:,:]
y_val = y_train[-701:]
x_train = x_train[:-701,:,:]
y_train = y_train[:-701]

In [7]:
# Print shapes of train, validation, and test sets
print(f'x_train shape:{x_train.shape},\
      \ny_train shape:{y_train.shape},\
      \nx_val shape:{x_val.shape},\
      \ny_val shape:{y_val.shape},\
      \nx_test shape:{x_test.shape},\
      \ny_test shape:{y_test.shape}\n')

# List the classes that exist within the dataset
print(f'Classes: {np.unique(y_train)}')
print('Seizure Types: \n1: for Complex Partial Seizures, 2: for Electrographic Seizures,\
       \n3: for Video-detected Seizures with no visual change over EEG, 0: for Normal data')

x_train shape:(5531, 19, 500),      
y_train shape:(5531,),      
x_val shape:(701, 19, 500),      
y_val shape:(701,),      
x_test shape:(1558, 19, 500),      
y_test shape:(1558,)

Classes: [0 1 2 3]
Seizure Types: 
1: for Complex Partial Seizures, 2: for Electrographic Seizures,       
3: for Video-detected Seizures with no visual change over EEG, 0: for Normal data


## Models

### Model : LSTM

In [8]:
# Build the neural network model

tf.random.set_seed(1234)                                   # For consistent results

model = Sequential([
    Input((x_train.shape[1],x_train.shape[2],1)),          # Input layer for EEG data: (electrodes, time points, channels)

    Conv2D(8, (1,250), activation='relu', padding='same'), # Kernel length set to half the sampling rate of the data
    BatchNormalization(),                                  # Normalize activations to stabilize training
    AveragePooling2D((1,4)),                               # Downsample feature maps to reduce computational complexity and prevent overfitting
    Dropout(0.2),                                          # Include dropout to prevent overfitting

    Conv2D(16,(1,62), activation='relu', padding='same'),  # Additional convolutional layers to capture hierarchical features
    BatchNormalization(),
    AveragePooling2D((1,2)),
    Dropout(0.2),

    Conv2D(32,(1,31), activation='relu', padding='same'),
    BatchNormalization(),
    AveragePooling2D((1,2)),
    Dropout(0.2),

    Reshape((x_train.shape[1],-1)),                        # Reshape to prepare for LSTM layer
    LSTM(128, activation='tanh'),

    Dense(4)
])

In [9]:
# Compile and fit the model

model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer = Adamax(0.001),
    metrics = ['accuracy']
)

model.fit(
    x_train,y_train,
    epochs = 20,
    verbose = 1,
    validation_data=(x_val,y_val)
)

train_loss, train_accuracy = model.evaluate(x_train, y_train)
val_loss, val_accuracy = model.evaluate(x_val, y_val)
test_loss, test_accuracy = model.evaluate(x_test, y_test)

print(f'Train Loss: {train_loss:.2f}, Validation Loss: {val_loss:.2f},  Test Loss: {test_loss:.2f}')
print(f'Train Accuracy: {train_accuracy*100:.2f}%, Validation Accuracy: {val_accuracy*100:.2f}%, Test Accuracy: {test_accuracy*100:.2f}%')

Epoch 1/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 12s 29ms/step - accuracy: 0.6876 - loss: 0.8082 - val_accuracy: 0.4950 - val_loss: 1.3881
Epoch 2/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.8105 - loss: 0.5152 - val_accuracy: 0.1170 - val_loss: 1.4843
Epoch 3/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.8619 - loss: 0.3769 - val_accuracy: 0.4051 - val_loss: 1.1780
Epoch 4/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.8926 - loss: 0.2961 - val_accuracy: 0.4351 - val_loss: 1.3384
Epoch 5/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9033 - loss: 0.2511 - val_accuracy: 0.8673 - val_loss: 0.3206
Epoch 6/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9202 - loss: 0.2141 - val_accuracy: 0.8930 - val_loss: 0.2628
Epoch 7/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.9255 - loss: 0.1860 - val_accuracy: 0.8830 - val_loss: 0.2698
Epoch 8/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9396 - loss: 0.1574 - val_ac

In [12]:
## Saving the model weights and archeture
model.save("model_01.h5")

### Model 2: 1D CNN + GRU Hybrid

In [21]:
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D

model2 = Sequential([
    Input((x_train.shape[1], x_train.shape[2])),

    Conv1D(64, kernel_size=7, padding='same', activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Conv1D(128, kernel_size=5, padding='same', activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    GRU(64, return_sequences=False),
    Dense(4)
])


In [22]:
model2.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(0.001),
    metrics=['accuracy']
)

model2.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=20,
    verbose=1
)

Epoch 1/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.6451 - loss: 0.9069 - val_accuracy: 0.5321 - val_loss: 1.0220
Epoch 2/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7392 - loss: 0.6580 - val_accuracy: 0.6605 - val_loss: 0.8256
Epoch 3/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7890 - loss: 0.5506 - val_accuracy: 0.7532 - val_loss: 0.5993
Epoch 4/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8346 - loss: 0.4494 - val_accuracy: 0.8074 - val_loss: 0.5040
Epoch 5/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.8605 - loss: 0.3911 - val_accuracy: 0.8245 - val_loss: 0.5034
Epoch 6/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8770 - loss: 0.3364 - val_accuracy: 0.8245 - val_loss: 0.5219
Epoch 7/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8868 - loss: 0.3004 - val_accuracy: 0.8060 - val_loss: 0.5692
Epoch 8/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9025 - loss: 0.2686 - val_accura

In [15]:
## saving the models
model2.save("model_02.h5")

### Model 3: Transformer-style Attention Model

In [23]:
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Add

input_layer = Input(shape=(x_train.shape[1], x_train.shape[2]))

# Positional Encoding not added for simplicity – you can try later
attn_output = MultiHeadAttention(num_heads=4, key_dim=32)(input_layer, input_layer)
attn_output = Dropout(0.2)(attn_output)
attn_output = Add()([input_layer, attn_output])  # Residual Connection
attn_output = LayerNormalization()(attn_output)

# Feed Forward Network
ffn = Dense(128, activation='relu')(attn_output)
ffn = Dropout(0.3)(ffn)
ffn = Dense(64, activation='relu')(ffn)
ffn = Dropout(0.2)(ffn)

flattened = Flatten()(ffn)
output_layer = Dense(4)(flattened)

model3 = tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [24]:
model3.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(0.001),
    metrics=['accuracy']
)

model3.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=20,
    verbose=1
)

Epoch 1/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 14s 32ms/step - accuracy: 0.6271 - loss: 0.8641 - val_accuracy: 0.7989 - val_loss: 0.5117
Epoch 2/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8009 - loss: 0.5110 - val_accuracy: 0.8345 - val_loss: 0.4511
Epoch 3/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8387 - loss: 0.4382 - val_accuracy: 0.8359 - val_loss: 0.4487
Epoch 4/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8581 - loss: 0.4031 - val_accuracy: 0.8488 - val_loss: 0.4181
Epoch 5/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8582 - loss: 0.3658 - val_accuracy: 0.8459 - val_loss: 0.4188
Epoch 6/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8573 - loss: 0.3726 - val_accuracy: 0.8631 - val_loss: 0.3813
Epoch 7/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8838 - loss: 0.3202 - val_accuracy: 0.8545 - val_loss: 0.4021
Epoch 8/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8765 - loss: 0.3311 - val_accuracy: 

In [27]:
## saving the model
model3.save("model3.h5")

## Evaluation Of Models

### Model : LSTM

In [19]:
# Evaluate the model with other metrics

# Train data
y_train_predict_ = model.predict(x_train)
y_train_predict = np.argmax(tf.nn.softmax(y_train_predict_), axis=1)

# Test data
y_test_predict_ = model.predict(x_test)
y_test_predict = np.argmax(tf.nn.softmax(y_test_predict_), axis=1)

# Define the metrics and corresponding values
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
train_values = [accuracy_score(y_train, y_train_predict) * 100,
                precision_score(y_train, y_train_predict, average='macro') * 100,
                recall_score(y_train, y_train_predict, average='macro') * 100,
                f1_score(y_train, y_train_predict, average='macro') * 100]
test_values = [accuracy_score(y_test, y_test_predict) * 100,
               precision_score(y_test, y_test_predict, average='macro') * 100,
               recall_score(y_test, y_test_predict, average='macro') * 100,
               f1_score(y_test, y_test_predict, average='macro') * 100]

# Create DataFrame
results_df = pd.DataFrame({
    'Metric:': metrics,
    'Train:': train_values,
    'Test:': test_values
})



# Print DataFrame
print("\nTrain and Test Data Results -")
print(results_df.round(2).to_string(index=False))

173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step

Train and Test Data Results -
  Metric:  Train:  Test:
 Accuracy   97.43  93.07
Precision   97.85  94.63
   Recall   98.02  92.85
 F1 Score   97.90  93.62


### Model 2: 1D CNN + GRU Hybrid

In [20]:
# Evaluate the model with other metrics

# Train data
y_train_predict_ = model2.predict(x_train)
y_train_predict = np.argmax(tf.nn.softmax(y_train_predict_), axis=1)

# Test data
y_test_predict_ = model2.predict(x_test)
y_test_predict = np.argmax(tf.nn.softmax(y_test_predict_), axis=1)

# Define the metrics and corresponding values
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
train_values = [accuracy_score(y_train, y_train_predict) * 100,
                precision_score(y_train, y_train_predict, average='macro') * 100,
                recall_score(y_train, y_train_predict, average='macro') * 100,
                f1_score(y_train, y_train_predict, average='macro') * 100]
test_values = [accuracy_score(y_test, y_test_predict) * 100,
               precision_score(y_test, y_test_predict, average='macro') * 100,
               recall_score(y_test, y_test_predict, average='macro') * 100,
               f1_score(y_test, y_test_predict, average='macro') * 100]

# Create DataFrame
results_df = pd.DataFrame({
    'Metric:': metrics,
    'Train:': train_values,
    'Test:': test_values
})



# Print DataFrame
print("\nTrain and Test Data Results -")
print(results_df.round(2).to_string(index=False))

173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

Train and Test Data Results -
  Metric:  Train:  Test:
 Accuracy   94.12  84.53
Precision   92.99  85.39
   Recall   89.58  71.83
 F1 Score   90.49  73.87


### Model 3: Transformer-style Attention Model

In [25]:
# Evaluate the model with other metrics

# Train data
y_train_predict_ = model3.predict(x_train)
y_train_predict = np.argmax(tf.nn.softmax(y_train_predict_), axis=1)

# Test data
y_test_predict_ = model3.predict(x_test)
y_test_predict = np.argmax(tf.nn.softmax(y_test_predict_), axis=1)

# Define the metrics and corresponding values
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
train_values = [accuracy_score(y_train, y_train_predict) * 100,
                precision_score(y_train, y_train_predict, average='macro') * 100,
                recall_score(y_train, y_train_predict, average='macro') * 100,
                f1_score(y_train, y_train_predict, average='macro') * 100]
test_values = [accuracy_score(y_test, y_test_predict) * 100,
               precision_score(y_test, y_test_predict, average='macro') * 100,
               recall_score(y_test, y_test_predict, average='macro') * 100,
               f1_score(y_test, y_test_predict, average='macro') * 100]

# Create DataFrame
results_df = pd.DataFrame({
    'Metric:': metrics,
    'Train:': train_values,
    'Test:': test_values
})



# Print DataFrame
print("\nTrain and Test Data Results -")
print(results_df.round(2).to_string(index=False))

173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step

Train and Test Data Results -
  Metric:  Train:  Test:
 Accuracy   92.95  86.39
Precision   94.36  88.94
   Recall   94.64  80.66
 F1 Score   94.42  83.69


In [28]:
!cp /content/model_01.h5 /content/drive/MyDrive/AdnanAminAssignment
!cp /content/model_02.h5 /content/drive/MyDrive/AdnanAminAssignment
!cp /content/model_03.h5 /content/drive/MyDrive/AdnanAminAssignment